In [5]:
#Packages
import os
import pandas as pd
import itertools
import numpy as np
from operator import itemgetter

In [6]:
shares = pd.read_csv('../temporary/moving_average_shares.csv', index_col='Unnamed: 0')
shares

,1510,1511,1512,1513,1514,1515,1516,1517,1518,1519,...,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890
1x2,0.000010,1.993856e-06,1.890401e-06,1.797580e-06,1.797580e-06,0.000025,0.000023,0.000023,0.000023,0.000023,...,0.002293,0.002321,0.002335,0.002333,0.002247,0.002226,0.002238,0.002245,0.002236,0.002221
1x3,0.000005,1.068301e-06,1.005507e-06,9.554093e-07,9.554093e-07,0.000012,0.000011,0.000011,0.000011,0.000011,...,0.000669,0.000691,0.000721,0.000737,0.000755,0.000758,0.000769,0.000787,0.000802,0.000786
1x4,0.000009,2.563470e-06,2.364779e-06,2.242189e-06,2.242189e-06,0.000020,0.000019,0.000019,0.000019,0.000019,...,0.000735,0.000739,0.000704,0.000693,0.000674,0.000659,0.000636,0.000626,0.000647,0.000630
1x5,0.000014,1.480231e-06,1.488265e-06,1.423555e-06,1.423555e-06,0.000212,0.000199,0.000199,0.000199,0.000199,...,0.001772,0.001853,0.001881,0.001907,0.001958,0.001996,0.002015,0.002068,0.002129,0.002141
1x6,0.000002,1.352189e-07,1.484902e-07,1.431995e-07,1.431995e-07,0.000005,0.000006,0.000006,0.000006,0.000006,...,0.000304,0.000328,0.000336,0.000359,0.000374,0.000373,0.000368,0.000363,0.000350,0.000349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57x59,0.000011,1.248851e-05,1.004193e-04,9.509673e-05,9.509673e-05,0.000084,0.000079,0.000079,0.000079,0.000079,...,0.000035,0.000035,0.000033,0.000033,0.000033,0.000033,0.000032,0.000033,0.000032,0.000033
57x60,0.000002,1.767700e-07,7.619845e-04,7.197240e-04,7.197240e-04,0.000632,0.000595,0.000595,0.000595,0.000595,...,0.000128,0.000133,0.000133,0.000142,0.000153,0.000153,0.000149,0.000148,0.000153,0.000155
58x59,0.003315,3.998958e-03,3.537325e-03,3.338580e-03,3.338580e-03,0.002930,0.002758,0.002758,0.002758,0.002758,...,0.000198,0.000202,0.000204,0.000199,0.000200,0.000204,0.000205,0.000209,0.000214,0.000214
58x60,0.000104,1.116548e-04,9.957842e-05,9.400082e-05,9.400082e-05,0.000086,0.000081,0.000081,0.000081,0.000081,...,0.000916,0.000920,0.000944,0.000950,0.000983,0.001013,0.001030,0.001046,0.001066,0.001057


In [7]:
def category_shares(data, year, pol, sci, rel):
    #'pol', 'sci', and 'rel' need to be lists of topics that have been designated as making up each category. This is the output of 'categories.py' and assignments to 'Religion', 'Science', and 'Political Economy' are ultimately subjective.
    
    shares = data[year]
    

In [8]:
def category_score(topic, cat):
    c = set(cat) #turn category into set
    c.update([topic]) #add topic to set
    cross_combos = list(itertools.combinations(c,2)) #creates combinations of cross-topics
    cross_combos = ['x'.join(map(str,i)) for i in cross_combos] #easy referencing for 'moving_average_shares'
    print(cross_combos)
    return cross_combos

In [11]:
num = 1
religion = [2,3,4]

ls = category_score(num, religion)


TypeError: unhashable type: 'list'

In [15]:
num = [1]

list(itertools.product(num, religion))

[(1, 2), (1, 3), (1, 4)]

In [ ]:
t =list(itemgetter(*ls)(moving_average_shares[1510]))
sum(t)